In [15]:
import os 
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

import os 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

root_path = os.getcwd()

/kaggle/input\jigsaw-toxic-comment-classification-challenge\sample_submission.csv.zip
/kaggle/input\jigsaw-toxic-comment-classification-challenge\test.csv.zip
/kaggle/input\jigsaw-toxic-comment-classification-challenge\test_labels.csv.zip
/kaggle/input\jigsaw-toxic-comment-classification-challenge\train.csv.zip


### **Load dataset**

In [16]:
%cd E:\Toxic-comment-classification\

E:\Toxic-comment-classification


In [17]:
train = pd.read_csv('kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test = pd.read_csv('kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
test_label = pd.read_csv('kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip')
sample_submission = pd.read_csv('kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')

In [18]:
def read_from_file(filename):
    with open(filename, 'r') as f:
        return np.array(f.read().splitlines())

In [19]:
# test.comment_text = read_from_file('clean_data/data_test_cleaned_light2.txt')
# train.comment_text = read_from_file('clean_data/data_train_cleaned_light2.txt')
test.comment_text = read_from_file('clean_data/data_test_cleaned_vanilla2.txt')
train.comment_text = read_from_file('clean_data/data_train_cleaned_vanilla2.txt')

In [20]:
test

,id,comment_text
0,00001cee341fdb12,yo bitch ja rule is more succesful then you wi...
1,0000247867823ef7,from rfc the title is fine as it is imo
2,00013b17ad220c46,sources zawe ashton on lapland
3,00017563c3f7919a,if you have a look back at the source the info...
4,00017695ad8997eb,i do not anonymously edit articles at all
...,...,...
153159,fffcd0960ee309b5,i totally agree this stuff is nothing but too ...
153160,fffd7a9a6eb32c16,throw from out field to home plate does it get...
153161,fffda9e8d6fafa9e,okinotorishima categories i see your changes a...
153162,fffe8f1340a79fc2,one of the founding nations of the eu germany ...


In [21]:
train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww he matches this background colour i am se...,0,0,0,0,0,0
2,000113f07ec002fd,hey man i am really not trying to edit war it ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,more i cannot make any real suggestions on imp...,0,0,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,and for the second time of asking when your vi...,0,0,0,0,0,0
159567,ffea4adeee384e90,you should be ashamed of yourself that is a ho...,0,0,0,0,0,0
159568,ffee36eab5c267c9,spitzer umm theres no actual article for prost...,0,0,0,0,0,0
159569,fff125370e4aaaf3,and it looks like it was actually you who put ...,0,0,0,0,0,0


In [22]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

### **KFold CrossValidation**

In [23]:
vec = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9, max_features=100000, strip_accents='unicode', use_idf=True, smooth_idf=True, sublinear_tf=True)
X_t = train['comment_text'].values
y = train[list_classes].values
X_te = test['comment_text'].values

num_folds = 10

scores = []
oof_predict = np.zeros((train.shape[0], 6))

predict = np.zeros((test.shape[0], 6))
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X_t):
    kfold_X_train, kfold_X_valid = X_t[train_index], X_t[test_index]
    kfold_y_train, kfold_y_valid = y[train_index], y[test_index]
    
    # Vectorize text data
    kfold_X_train_vec = vec.fit_transform(kfold_X_train)
    kfold_X_valid_vec = vec.transform(kfold_X_valid)
    X_te_vec = vec.transform(X_te)
    
    # Initialize an empty list to store models for each label
    models = []
    
    # Train a separate Naive Bayes model for each label
    for i in range(y.shape[1]):
        model = MultinomialNB(alpha=0.035)
        model.fit(kfold_X_train_vec, kfold_y_train[:, i])
        models.append(model)
    
    predict += np.array([model.predict_proba(X_te_vec)[:, 1] for model in models]).T / num_folds
    
    oof_predict[test_index] = np.array([model.predict_proba(kfold_X_valid_vec)[:, 1] for model in models]).T

print('Done')

sample_submission[list_classes] = predict
sample_submission.to_csv('kaggle/working/' + '8_sub.csv', index=False)

oof = pd.DataFrame.from_dict({'id': train['id']})
for c in list_classes:
    oof[c] = np.zeros(len(train))  
oof[list_classes] = oof_predict
oof.to_csv('kaggle/working/' + '8_oof.csv', index=False)

Done
